# Addding Graph Structures to a SQL Server Database

This is the notebook to walk through the setup for my talk.

## Creating a database

This first section will create a database for the talk. The default name here is IMDB

In [0]:
CREATE DATABASE IMDB

Once the database is created, we can change to this database for the rest of the notebook

In [1]:
Use IMDB

Commands completed successfully.

Total execution time: 00:00:00.001

# Table Setup
The first queries here are designed to create the data structures used for the rest of the talk

This requires a SQL Server 2019 version

In [0]:
create table dbo.Movie (
  MovieID int not null IDENTITY(1,1) CONSTRAINT MoviePK PRIMARY Key
, MovieTitle varchar(200)
, MovieDescription varchar(1000)
, MovieYear int
, MovieRuntime smallint
) as Node;
GO
create table dbo.Person (
PersonID int not null IDENTITY(1,1) CONSTRAINT PersonPK PRIMARY KEY
, PersonName varchar(100)
) as Node;
GO
create TABLE dbo.Genre (
 GenreID int not null IDENTITY(1,1) CONSTRAINT GenrePK PRIMARY KEY
 , GenreName varchar(30)
) as Node;


## Loading Data from the CSV
This code assumes you have downloaded a CSV of data from 

This code requires [dbatools](https://dbatools.io) installed on your machine and a PowerShell prompt.

Run this code from the location of the CSV file or provide a full path. This will auto create the dbo.[IMDB-Movie-Data] table inside the database. You can access the contents of the $movie variable to confirm this works.

In [0]:
$movie = Import-DbaCsv -SqlInstance localhost -Database IMDB -AutoCreateTable -Path "IMDB-Movie-Data.csv"

## The ETL Process

This sections contains the code I used to move data from the table, dbo.[IMDB-Movie-Data], which is created in the last step.

The first part of this is to move the movie information from the staging table

In [0]:
insert dbo.Movie (MovieTitle, MovieDescription, MovieYear, MovieRuntime)
 select m.title, m.description, m.year, m.[runtime (Minutes)]
 from dbo.[IMDB-Movie-data] m


You can check that this succeeded with this query:

In [2]:
select top 20 * from dbo.Movie

(20 rows affected)

Total execution time: 00:00:00.020

$node_id_25E35AF55DAF474DB21E7E851495EDD1,MovieID,MovieTitle,MovieDescription,MovieYear,MovieRuntime
"{""type"":""node"",""schema"":""dbo"",""table"":""Movie"",""id"":0}",1,Guardians of the Galaxy,A group of intergalactic criminals are forced to work together to stop a fanatical warrior from taking control of the universe.,2014,121
"{""type"":""node"",""schema"":""dbo"",""table"":""Movie"",""id"":1}",2,Prometheus,"Following clues to the origin of mankind, a team finds a structure on a distant moon, but they soon realize they are not alone.",2012,124
"{""type"":""node"",""schema"":""dbo"",""table"":""Movie"",""id"":2}",3,Split,Three girls are kidnapped by a man with a diagnosed 23 distinct personalities. They must try to escape before the apparent emergence of a frightful new 24th.,2016,117
"{""type"":""node"",""schema"":""dbo"",""table"":""Movie"",""id"":3}",4,Sing,"In a city of humanoid animals, a hustling theater impresario's attempt to save his theater with a singing competition becomes grander than he anticipates even as its finalists' find that their lives will never be the same.",2016,108
"{""type"":""node"",""schema"":""dbo"",""table"":""Movie"",""id"":4}",5,Suicide Squad,A secret government agency recruits some of the most dangerous incarcerated super-villains to form a defensive task force. Their first mission: save the world from the apocalypse.,2016,123
"{""type"":""node"",""schema"":""dbo"",""table"":""Movie"",""id"":5}",6,The Great Wall,European mercenaries searching for black powder become embroiled in the defense of the Great Wall of China against a horde of monstrous creatures.,2016,103
"{""type"":""node"",""schema"":""dbo"",""table"":""Movie"",""id"":6}",7,La La Land,A jazz pianist falls for an aspiring actress in Los Angeles.,2016,128
"{""type"":""node"",""schema"":""dbo"",""table"":""Movie"",""id"":7}",8,Mindhorn,"A has-been actor best known for playing the title character in the 1980s detective series ""Mindhorn"" must work with the police when a serial killer says that he will only speak with Detective Mindhorn, whom he believes to be a real person.",2016,89
"{""type"":""node"",""schema"":""dbo"",""table"":""Movie"",""id"":8}",9,The Lost City of Z,"A true-life drama, centering on British explorer Col. Percival Fawcett, who disappeared while searching for a mysterious city in the Amazon in the 1920s.",2016,141
"{""type"":""node"",""schema"":""dbo"",""table"":""Movie"",""id"":9}",10,Passengers,"A spacecraft traveling to a distant colony planet and transporting thousands of people has a malfunction in its sleep chambers. As a result, two passengers are awakened 90 years early.",2016,116


## Loading Genres

We want to load genres, but we want a distinct list of them. Let's examine the data.

In [5]:
select top 10 * from dbo.[IMDB-Movie-Data]

(10 rows affected)

Total execution time: 00:00:00.014

Rank,Title,Genre,Description,Director,Actors,Year,Runtime (Minutes),Rating,Votes,Revenue (Millions),Metascore
1,Guardians of the Galaxy,"Action,Adventure,Sci-Fi",A group of intergalactic criminals are forced to work together to stop a fanatical warrior from taking control of the universe.,James Gunn,"Chris Pratt, Vin Diesel, Bradley Cooper, Zoe Saldana",2014,121,8.1,757074,333.13,76
2,Prometheus,"Adventure,Mystery,Sci-Fi","Following clues to the origin of mankind, a team finds a structure on a distant moon, but they soon realize they are not alone.",Ridley Scott,"Noomi Rapace, Logan Marshall-Green, Michael Fassbender, Charlize Theron",2012,124,7,485820,126.46,65
3,Split,"Horror,Thriller",Three girls are kidnapped by a man with a diagnosed 23 distinct personalities. They must try to escape before the apparent emergence of a frightful new 24th.,M. Night Shyamalan,"James McAvoy, Anya Taylor-Joy, Haley Lu Richardson, Jessica Sula",2016,117,7.3,157606,138.12,62
4,Sing,"Animation,Comedy,Family","In a city of humanoid animals, a hustling theater impresario's attempt to save his theater with a singing competition becomes grander than he anticipates even as its finalists' find that their lives will never be the same.",Christophe Lourdelet,"Matthew McConaughey,Reese Witherspoon, Seth MacFarlane, Scarlett Johansson",2016,108,7.2,60545,270.32,59
5,Suicide Squad,"Action,Adventure,Fantasy",A secret government agency recruits some of the most dangerous incarcerated super-villains to form a defensive task force. Their first mission: save the world from the apocalypse.,David Ayer,"Will Smith, Jared Leto, Margot Robbie, Viola Davis",2016,123,6.2,393727,325.02,40
6,The Great Wall,"Action,Adventure,Fantasy",European mercenaries searching for black powder become embroiled in the defense of the Great Wall of China against a horde of monstrous creatures.,Yimou Zhang,"Matt Damon, Tian Jing, Willem Dafoe, Andy Lau",2016,103,6.1,56036,45.13,42
7,La La Land,"Comedy,Drama,Music",A jazz pianist falls for an aspiring actress in Los Angeles.,Damien Chazelle,"Ryan Gosling, Emma Stone, Rosemarie DeWitt, J.K. Simmons",2016,128,8.3,258682,151.06,93
8,Mindhorn,Comedy,"A has-been actor best known for playing the title character in the 1980s detective series ""Mindhorn"" must work with the police when a serial killer says that he will only speak with Detective Mindhorn, whom he believes to be a real person.",Sean Foley,"Essie Davis, Andrea Riseborough, Julian Barratt,Kenneth Branagh",2016,89,6.4,2490,NULL,71
9,The Lost City of Z,"Action,Adventure,Biography","A true-life drama, centering on British explorer Col. Percival Fawcett, who disappeared while searching for a mysterious city in the Amazon in the 1920s.",James Gray,"Charlie Hunnam, Robert Pattinson, Sienna Miller, Tom Holland",2016,141,7.1,7188,8.01,78
10,Passengers,"Adventure,Drama,Romance","A spacecraft traveling to a distant colony planet and transporting thousands of people has a malfunction in its sleep chambers. As a result, two passengers are awakened 90 years early.",Morten Tyldum,"Jennifer Lawrence, Chris Pratt, Michael Sheen,Laurence Fishburne",2016,116,7,192177,100.01,41


If we look at the results above, we can see the genre column contains a comma separated list of values. This isn't what we want. We want each of these normalized into rows.

The first code is here to show you the query. We can use string_Split() to separate the data into a long list. This function requires a string to split. In this case, we use CROSS APPLY to take all the values from the original genre column and pass them in. We use the comma as the second parameter. 

Once this is done, we use a DISTINCT as we don't need multiple copies of each genre; just one.

In [4]:
select distinct g.value
 from dbo.[IMDB-Movie-Data] m
  cross apply string_split(m.genre, ',') g
GO

: Msg 207, Level 16, State 1, Line 2
Invalid column name ','.

: Msg 8116, Level 16, State 1, Line 1
Argument data type void type is invalid for argument 2 of string_split function.

Total execution time: 00:00:00.034

Let's insert this data now, using the query above with an insert.

In [0]:
insert Genre (GenreName)
select distinct g.value
 from dbo.[IMDB-Movie-Data] m
  cross apply string_split(m.genre, ',') g


Let's check that the insert completed successfully:

In [6]:
select * from dbo.genre

(20 rows affected)

Total execution time: 00:00:00.017

$node_id_91678B04D5AD42BE8147FB9AA69ADD29,GenreID,GenreName
"{""type"":""node"",""schema"":""dbo"",""table"":""Genre"",""id"":0}",1,Crime
"{""type"":""node"",""schema"":""dbo"",""table"":""Genre"",""id"":1}",2,Adventure
"{""type"":""node"",""schema"":""dbo"",""table"":""Genre"",""id"":2}",3,Animation
"{""type"":""node"",""schema"":""dbo"",""table"":""Genre"",""id"":3}",4,Comedy
"{""type"":""node"",""schema"":""dbo"",""table"":""Genre"",""id"":4}",5,Western
"{""type"":""node"",""schema"":""dbo"",""table"":""Genre"",""id"":5}",6,Fantasy
"{""type"":""node"",""schema"":""dbo"",""table"":""Genre"",""id"":6}",7,Music
"{""type"":""node"",""schema"":""dbo"",""table"":""Genre"",""id"":7}",8,Sci-Fi
"{""type"":""node"",""schema"":""dbo"",""table"":""Genre"",""id"":8}",9,Musical
"{""type"":""node"",""schema"":""dbo"",""table"":""Genre"",""id"":9}",10,Thriller


There are only 20 genres in total.

## Loading Peopple

We have two sets of people in our original dataset. I'll use the query below to show these.

In [8]:
select director, actors
 from dbo.[imdb-movie-data]

(1000 rows affected)

Total execution time: 00:00:00.032

director,actors
James Gunn,"Chris Pratt, Vin Diesel, Bradley Cooper, Zoe Saldana"
Ridley Scott,"Noomi Rapace, Logan Marshall-Green, Michael Fassbender, Charlize Theron"
M. Night Shyamalan,"James McAvoy, Anya Taylor-Joy, Haley Lu Richardson, Jessica Sula"
Christophe Lourdelet,"Matthew McConaughey,Reese Witherspoon, Seth MacFarlane, Scarlett Johansson"
David Ayer,"Will Smith, Jared Leto, Margot Robbie, Viola Davis"
Yimou Zhang,"Matt Damon, Tian Jing, Willem Dafoe, Andy Lau"
Damien Chazelle,"Ryan Gosling, Emma Stone, Rosemarie DeWitt, J.K. Simmons"
Sean Foley,"Essie Davis, Andrea Riseborough, Julian Barratt,Kenneth Branagh"
James Gray,"Charlie Hunnam, Robert Pattinson, Sienna Miller, Tom Holland"
Morten Tyldum,"Jennifer Lawrence, Chris Pratt, Michael Sheen,Laurence Fishburne"


I limited the data, but I need to be concerned about a couple things. We can see in the Actors column that there are multiple people stored in this column. We'll see to ensure we split this data and add each distinct row to the table. However, the other issue is that I was worried that the Director column contained multiple directors for a movie. I decided to run this query to check:

In [9]:
select *
 from dbo.[IMDB-Movie-Data]
  where charindex(',',Director) > 0

(0 rows affected)

Total execution time: 00:00:00.012

Rank,Title,Genre,Description,Director,Actors,Year,Runtime (Minutes),Rating,Votes,Revenue (Millions),Metascore


I didn't get any results, so this leads me to believe that none of the rows in the director section contain more than one value.